In [1]:
import pandas as pd

In [2]:
income_statement = pd.read_json("https://fmpcloud.io/api/v3/income-statement/AAPL?limit=120&apikey=54add230ef61c63c8896ae8597c99d5c")
income_statement.set_index('date', inplace=True)

In [3]:
cash_flow = pd.read_json("https://fmpcloud.io/api/v3/cash-flow-statement/AAPL?limit=120&apikey=54add230ef61c63c8896ae8597c99d5c")
cash_flow.set_index('date', inplace=True)

In [4]:
data=income_statement[['revenue','eps']].copy()

In [5]:
data['fcf']=cash_flow['freeCashFlow']

In [6]:
data.head()

,revenue,eps,fcf
date,,,
2021-09-25,365817000000,5.6700,92953000000
2020-09-26,274515000000,3.3100,73365000000
2019-09-28,260174000000,2.9925,58896000000
2018-09-29,265595000000,3.0025,64121000000
2017-09-30,229234000000,2.3175,50803000000


In [7]:
data=data.sort_index()

In [8]:
data.head()

,revenue,eps,fcf
date,,,
2017-09-30,229234000000,2.3175,50803000000
2018-09-29,265595000000,3.0025,64121000000
2019-09-28,260174000000,2.9925,58896000000
2020-09-26,274515000000,3.3100,73365000000
2021-09-25,365817000000,5.6700,92953000000


In [9]:
data['rev%-chng']=data['revenue'].pct_change()
data['eps%-chng']=data['eps'].pct_change()
data['fcf%-chng']=data['fcf'].pct_change()
data.head()

,revenue,eps,fcf,rev%-chng,eps%-chng,fcf%-chng
date,,,,,,
2017-09-30,229234000000,2.3175,50803000000,NaN,NaN,NaN
2018-09-29,265595000000,3.0025,64121000000,0.158620,0.295577,0.262150
2019-09-28,260174000000,2.9925,58896000000,-0.020411,-0.003331,-0.081487
2020-09-26,274515000000,3.3100,73365000000,0.055121,0.106099,0.245670
2021-09-25,365817000000,5.6700,92953000000,0.332594,0.712991,0.266994


In [10]:
#Cortamos las fechas
data_set = data.loc['2018-09-29':]
data_set.index=data_set.index.year
data_set

,revenue,eps,fcf,rev%-chng,eps%-chng,fcf%-chng
date,,,,,,
2018,265595000000,3.0025,64121000000,0.158620,0.295577,0.262150
2019,260174000000,2.9925,58896000000,-0.020411,-0.003331,-0.081487
2020,274515000000,3.3100,73365000000,0.055121,0.106099,0.245670
2021,365817000000,5.6700,92953000000,0.332594,0.712991,0.266994


In [11]:
rev = data_set[['revenue','rev%-chng']].copy()
eps = data_set[['eps','eps%-chng']].copy()
fcf = data_set[['fcf','fcf%-chng']].copy()

In [12]:
def create_sheet(writer, df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name)
writer = pd.ExcelWriter("financials.xlsx", engine='xlsxwriter')
workbook = writer.book
create_sheet(writer, rev, 'Revenue')
create_sheet(writer, eps, 'EPS')
create_sheet(writer, fcf, 'FCF')
workbook.close()

In [13]:
#funcion para crear hojas en excel
def create_sheet(writer, df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name)
    worksheet = writer.sheets[sheet_name]
    workbook = writer.book
      
    chart = workbook.add_chart({'type': 'line'})
    
    chart.add_series({
        'name': sheet_name,
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 1, 11, 1],
    })
    
    column_chart = workbook.add_chart({'type': 'column'})
    
    column_chart.add_series({
        'name': sheet_name,
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 2, 11, 2],
        'y2_axis': True,
    })
    
    chart.combine(column_chart)
    chart.set_title({'name': sheet_name})
    chart.set_x_axis({'name': 'Date'})
    chart.set_y_axis({'name': 'Value'})
    column_chart.set_y2_axis({'name': '%-chg'})
    
    worksheet.insert_chart("E1", chart)
writer = pd.ExcelWriter("financials.xlsx", engine='xlsxwriter')
workbook = writer.book
create_sheet(writer, rev, 'Revenue')
create_sheet(writer, eps, 'EPS')
create_sheet(writer, fcf, 'FCF')
workbook.close()
writer = pd.ExcelWriter("financials.xlsx", engine='xlsxwriter')
workbook = writer.book
create_sheet(writer, rev, 'Revenue')
create_sheet(writer, eps, 'EPS')
create_sheet(writer, fcf, 'FCF')
workbook.close()